## Natural Language processing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [3]:
dataset=pd.read_csv('./../0. DataSets/Restaurant_Reviews.tsv',delimiter='\t',quoting=3)

In [5]:
dataset.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [8]:
review=re.sub('[^a-zA-Z]'," ",dataset['Review'][0])
review=review.lower()

review

'wow    loved this place '